In [3]:
import pandas as pd
import numpy as np
from skimage.measure import label, regionprops, regionprops_table
import trackpy as tp
import matplotlib as mpl
import matplotlib.pyplot as plt
from os.path import isfile, join
import datetime
from tifffile import imread
from numba import jit, prange

This was an old version of particle tracking- the function of this file is wrapped into xpeem_batchv3

In [4]:
'''
batch process the segmented tiles of the image and just get the data
'''

minparticlesize = 5
maxparticlesize = 200
search_range = 5        # how many pixels the particles are allowed to jump between frames- 15 for dark regions and 3 for bright crox particles
mintracklength = 15     #filtering by track lengths of 15 frames works well for the dark regions, but for the light particles a smaller number is better
mypath = 'mov2_median_8000.tif'

In [29]:

def findparticles_3d_img(frames: np.array, minsize=5, maxsize = 200):
    """
        This function takes a 3d image "frames" with the structure [frame, x, y]

       Create a pandas dataframe, and then find the particles (regions) using skimage's label function.  Label connects
       regions of the same integer value, i.e. segmented regions. In this dataframe, I also save the perimeter, filled fraction,
       and the area.
    """
    features = pd.DataFrame()
    for i in range(frames.shape[0]):
        label_image = label(frames[i,:,:], background=0)
        props = regionprops_table(label_image,properties=("centroid", "area"))
        framedata = pd.DataFrame(props)
        framedata['frame'] = i
        features = features.append(framedata)
        print(datetime.datetime.now(), i)

    return features

In [27]:
segmented = imread(mypath)
features = findparticles_3d_img(segmented, minsize=5, maxsize = 50)     #minsize of 5 for the crox particles, and 15 for the dark regions
features.to_csv("mov2_median_8000.csv")

2023-09-28 18:54:40.819613 0
2023-09-28 18:54:40.910561 1
2023-09-28 18:54:41.006774 2
2023-09-28 18:54:41.203076 3
2023-09-28 18:54:41.515913 4
2023-09-28 18:54:41.680220 5
2023-09-28 18:54:41.828797 6
2023-09-28 18:54:41.953428 7
2023-09-28 18:54:42.066893 8
2023-09-28 18:54:42.204647 9
2023-09-28 18:54:42.330038 10
2023-09-28 18:54:42.461401 11
2023-09-28 18:54:42.604851 12
2023-09-28 18:54:42.719570 13
2023-09-28 18:54:42.818371 14
2023-09-28 18:54:42.923410 15
2023-09-28 18:54:43.017655 16
2023-09-28 18:54:43.137582 17
2023-09-28 18:54:43.233410 18
2023-09-28 18:54:43.323968 19
2023-09-28 18:54:43.508625 20
2023-09-28 18:54:43.606650 21
2023-09-28 18:54:43.702392 22
2023-09-28 18:54:43.803797 23
2023-09-28 18:54:43.894625 24
2023-09-28 18:54:43.988759 25
2023-09-28 18:54:44.095512 26
2023-09-28 18:54:44.186546 27
2023-09-28 18:54:44.276543 28
2023-09-28 18:54:44.374654 29
2023-09-28 18:54:44.465327 30
2023-09-28 18:54:44.559376 31
2023-09-28 18:54:44.668522 32
2023-09-28 18:54:44.

In [ ]:
master = pd.DataFrame()
t = tp.link_df(features, search_range, memory=25)
t1 = tp.filter_stubs(t, mintracklength)      #filtering by track lengths of 15 frames works well for the dark regions, but for the light particles a smaller number is better

master = master.append(t1)


In [20]:
master.to_csv('median_8000_masked.csv')